### Задание
ДЗ - в рамках ранее выполненной работы по обработке данных, выполнить все работы с пандас дата фреймом через SQL запросы. 
- разбить выборку на обучающую и тестовую
- проанализировать пропуски и решить, что с ними делать
- проанализировать выбросы
- создать/ удалить переменные
- закодировать категориальные переменные
- нормализовать числовые переменные (при необходимости)

### Загрузим данные и посмотрим что они из себя представляют

In [22]:
import pandas as pd
credit_df = pd.read_csv('credit_train.csv', encoding='WINDOWS-1251', on_bad_lines='skip', sep=';', decimal=',')
print(credit_df.head())
print("\nКоличество строк и столбцов:")
print(credit_df.shape)

   client_id gender   age marital_status job_position  credit_sum  \
0          1      M   NaN            NaN          UMN    59998.00   
1          2      F   NaN            MAR          UMN    10889.00   
2          3      M  32.0            MAR          SPC    10728.00   
3          4      F  27.0            NaN          SPC    12009.09   
4          5      M  45.0            NaN          SPC         NaN   

   credit_month tariff_id  score_shk education        living_region  \
0            10       1.6        NaN       GRD   КРАСНОДАРСКИЙ КРАЙ   
1             6       1.1        NaN       NaN               МОСКВА   
2            12       1.1        NaN       NaN      ОБЛ САРАТОВСКАЯ   
3            12       1.1        NaN       NaN    ОБЛ ВОЛГОГРАДСКАЯ   
4            10       1.1   0.421385       SCH  ЧЕЛЯБИНСКАЯ ОБЛАСТЬ   

   monthly_income  credit_count  overdue_credit_count  open_account_flg  
0         30000.0           1.0                   1.0                 0  
1         

### Разобьем выборку на обучающую и тестовую.

In [23]:
from sklearn.model_selection import train_test_split

credit_df_train, credit_df_test = train_test_split(credit_df, test_size=0.1, random_state=0)

### Получим список всех стлбцов и посчитаем количество битых запией в каждом.

In [24]:
import sqlite3

conn = sqlite3.connect(':memory:')

credit_df_train.set_index('client_id', inplace=True)
credit_df_train.to_sql('credit_train', conn, index=True, index_label='client_id', if_exists='replace')
columns = credit_df_train.columns
null_counts_list = []

for column in columns:
    query = f"SELECT COUNT(*) - COUNT({column}) AS null_count FROM credit_train"
    result = pd.read_sql_query(query, conn)
    null_counts_list.append({'column_name': column, 'null_count': result.iloc[0]['null_count']})

null_counts = pd.DataFrame(null_counts_list)

print(null_counts)

             column_name  null_count
0                 gender           0
1                    age           3
2         marital_status           3
3           job_position           0
4             credit_sum           2
5           credit_month           0
6              tariff_id           0
7              score_shk           7
8              education           5
9          living_region         179
10        monthly_income           4
11          credit_count        8332
12  overdue_credit_count        8332
13      open_account_flg           0


Как видно даже на первых 4 результатах данные очень плохие. Есть пропуски и несогласованность данных. Удалим все строки где есть пропуски. Как будет видно далее их чуть более 5%, что не критично.

In [25]:
conditions = " AND ".join([f"{col} IS NOT NULL" for col in columns])
delete_query = f"DELETE FROM credit_train WHERE NOT ({conditions})"
conn.execute(delete_query)
conn.commit()

cleaned_data = pd.read_sql_query("SELECT * FROM credit_train", conn)
print(cleaned_data.shape)

(145165, 15)


### Преобразуем столбцы gender, marital_status, job_position, education в числовые значения

In [26]:
convert_to_numerical = """
UPDATE credit_train
SET gender = CASE gender
    WHEN 'M' THEN 1
    WHEN 'F' THEN 0
    ELSE -1
END,
marital_status = CASE marital_status
    WHEN 'MAR' THEN 1
    WHEN 'UNM' THEN 0
    ELSE -1
END,
job_position = CASE job_position
    WHEN 'SPC' THEN 1
    WHEN 'UMN' THEN 0
    ELSE -1
END,
education = CASE education
    WHEN 'GRD' THEN 1
    WHEN 'SCH' THEN 0
    ELSE -1
END;
"""

conn.execute(convert_to_numerical)
conn.commit()
converted_data = pd.read_sql_query("SELECT gender, marital_status, job_position, education FROM credit_train LIMIT 5", conn)
print(converted_data)

  gender marital_status job_position education
0      1              1            1         0
1      0              1            1         0
2      0             -1            1         1
3      0              0            1         0
4      0              0            1         1


### Создадим отдельный столбец обозначающий, что человек из Питерской или Московской области. Удалим столбец living_region

In [27]:
# Добавляем новый столбец
conn.execute("""
ALTER TABLE credit_train ADD COLUMN is_moscow_or_piter INTEGER;
""")

# Обновляем значения в новом столбце
conn.execute("""
UPDATE credit_train
SET is_moscow_or_piter = CASE
WHEN living_region LIKE '%МОСК%' OR living_region LIKE '%ПЕТЕР%' THEN 1
ELSE 0
END;
""")

# Удаляем старый столбец
conn.execute("""
ALTER TABLE credit_train DROP COLUMN living_region;
""")

conn.commit()
converted_data = pd.read_sql_query("SELECT * FROM credit_train LIMIT 5", conn)

print(converted_data)

   client_id gender   age marital_status job_position  credit_sum  \
0      76594      1  30.0              1            1     17869.0   
1     149378      0  38.0              1            1     33225.0   
2      47222      0  56.0             -1            1     57477.0   
3      14521      0  60.0              0            1     30598.0   
4      93568      0  26.0              0            1     27388.0   

   credit_month tariff_id  score_shk education  monthly_income  credit_count  \
0            10       1.1   0.352364         0         40000.0           0.0   
1            12       1.6   0.635078         0         20000.0           0.0   
2            10       1.6   0.513687         1        105000.0           6.0   
3            18       1.1   0.569651         0         30500.0           4.0   
4            10       1.1   0.376575         1         43000.0           2.0   

   overdue_credit_count  open_account_flg  is_moscow_or_piter  
0                   0.0                 

### Нормализуем данные в столбцах, где это необходимо

In [28]:
columns_to_normalize = ['age', 'credit_sum', 'credit_month', 'tariff_id', 'monthly_income', 'credit_count']

def normalize_column(conn, table, column):
    min_value = conn.execute(f"SELECT MIN({column}) FROM {table}").fetchone()[0]
    max_value = conn.execute(f"SELECT MAX({column}) FROM {table}").fetchone()[0]
    conn.execute(f"""
    UPDATE {table}
    SET {column} = ({column} - {min_value}) / ({max_value} - {min_value})
    """)
    conn.commit()


for column in columns_to_normalize:
    normalize_column(conn, 'credit_train', column)

normalized_data = pd.read_sql_query("SELECT * FROM credit_train LIMIT 5", conn)
print(normalized_data)

   client_id gender       age marital_status job_position  credit_sum  \
0      76594      1  0.226415              1            1    0.076714   
1     149378      0  0.377358              1            1    0.154559   
2      47222      0  0.716981             -1            1    0.277501   
3      14521      0  0.792453              0            1    0.141242   
4      93568      0  0.150943              0            1    0.124970   

   credit_month          tariff_id  score_shk education  monthly_income  \
0             0  0.104166666666667   0.352364         0        0.037037   
1             0              0.625   0.635078         0        0.015873   
2             0              0.625   0.513687         1        0.105820   
3             0  0.104166666666667   0.569651         0        0.026984   
4             0  0.104166666666667   0.376575         1        0.040212   

   credit_count  overdue_credit_count  open_account_flg  is_moscow_or_piter  
0      0.000000                 

### Удалим строки с выбросами по столбцу credit_sum

In [30]:
credit_sum_df = pd.read_sql_query("SELECT credit_sum FROM credit_train", conn)
Q1 = credit_sum_df['credit_sum'].quantile(0.25)
Q3 = credit_sum_df['credit_sum'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

delete_outliers_query = f"""
DELETE FROM credit_train
WHERE credit_sum < {lower_bound} OR credit_sum > {upper_bound}
"""
conn.execute(delete_outliers_query)
conn.commit()

# Проверка результата
cleaned_data = pd.read_sql_query("SELECT * FROM credit_train LIMIT 5", conn)
print(cleaned_data)

   client_id gender       age marital_status job_position  credit_sum  \
0      76594      1  0.226415              1            1    0.076714   
1     149378      0  0.377358              1            1    0.154559   
2      14521      0  0.792453              0            1    0.141242   
3      93568      0  0.150943              0            1    0.124970   
4     153915      0  0.566038             -1            1    0.097798   

   credit_month          tariff_id  score_shk education  monthly_income  \
0             0  0.104166666666667   0.352364         0        0.037037   
1             0              0.625   0.635078         0        0.015873   
2             0  0.104166666666667   0.569651         0        0.026984   
3             0  0.104166666666667   0.376575         1        0.040212   
4             0              0.625   0.545132         1        0.042328   

   credit_count  overdue_credit_count  open_account_flg  is_moscow_or_piter  
0      0.000000                 

### Выводы

Мы познакомились с основами SQL, освоили создание таблиц, основными типами запросов на выборку и обновление данных и управление столбцами. Однако, столкнулись и с определенными ограничениями, присущими базе данных SQLite3. Например, SQLite3 не предоставляет возможности работы с метаданными таблиц, не поддерживает подвыборки и не обладает множеством предопределенных функций для обработки данных, которые доступны в более продвинутых системах управления базами данных, таких как PostgreSQL.